In [1]:
import requests 
from bs4 import BeautifulSoup 
import pandas as pd

In [2]:
seasons=['https://fbref.com/en/comps/9/2020-2021/2020-2021-Premier-League-Stats',
         'https://fbref.com/en/comps/9/2021-2022/2021-2022-Premier-League-Stats',
         'https://fbref.com/en/comps/9/Premier-League-Stats']

In [3]:
dfs=[]

In [4]:
for season in seasons : 
    data=requests.get(season)
    src=data.content
    soup=BeautifulSoup(src)
    season_name=soup.find('h1')
    season_name=season_name.text.strip().split()[0]
    teams_table=soup.find('table',{'class':'stats_table'})
    teams_links=teams_table.find_all('a')
    teams_links=[l['href'] for l in teams_links]
    teams_links=[l for l in teams_links if '/squads' in l]
    teams_links=[f'https://fbref.com{l}'for l in teams_links]
    
    import time


    for link in teams_links : 
        data=requests.get(link) 
        src=data.content
        soup=BeautifulSoup(src) 
        matches=pd.read_html(src,match='Scores & Fixtures')
        matches=matches[0]
        shoots_url=soup.find_all('a')
        shoots_url=[l.get('href') for l in shoots_url] 
        shoots_url=[l for l in shoots_url if l and 'all_comps/shooting/' in l] 
        shoots_url=f'https://fbref.com/{shoots_url[0]}' 
        result=requests.get(shoots_url) 
        source=result.content 
        Soup=BeautifulSoup(source)
        shoots=pd.read_html(source,match='Shooting')
        shoots=shoots[0].drop(shoots[0].index[-1]) 
        shoots.columns=shoots.columns.droplevel(level=0)
        try : 
            team_data=matches.merge(shoots[['Date','Gls','Sh','SoT','SoT%','G/Sh','G/SoT','FK','PK','PKatt','xG','npxG','npxG/Sh','G-xG','np:G-xG']],on='Date')
        except ValueError : 
            continue    
        team_data=team_data[team_data['Comp']=='Premier League'] 
        team_name=link.split('/')[-1].replace('-Stats','').replace('-','')
        team_data['Team']=team_name
        team_data['Season']=season_name
        dfs.append(team_data)
        time.sleep(2)

In [6]:
dfs[0].head()

,Date,Time,Comp,Round,Day,Venue,Result,GF,GA,Opponent,...,FK,PK,PKatt,xG_y,npxG,npxG/Sh,G-xG,np:G-xG,Team,Season
0,2020-09-21,20:15,Premier League,Matchweek 2,Mon,Away,W,3,1,Wolves,...,2.0,1,1,1.9,1.1,0.09,1.1,0.9,ManchesterCity,2020-2021
2,2020-09-27,16:30,Premier League,Matchweek 3,Sun,Home,L,2,5,Leicester City,...,1.0,0,0,0.9,0.9,0.05,1.1,1.1,ManchesterCity,2020-2021
4,2020-10-03,17:30,Premier League,Matchweek 4,Sat,Away,D,1,1,Leeds United,...,1.0,0,0,1.2,1.2,0.05,-0.2,-0.2,ManchesterCity,2020-2021
5,2020-10-17,17:30,Premier League,Matchweek 5,Sat,Home,W,1,0,Arsenal,...,0.0,0,0,1.3,1.3,0.11,-0.3,-0.3,ManchesterCity,2020-2021
7,2020-10-24,12:30,Premier League,Matchweek 6,Sat,Away,D,1,1,West Ham,...,1.0,0,0,1.0,1.0,0.07,0.0,0.0,ManchesterCity,2020-2021


In [7]:
premier_league=pd.concat(dfs,axis=0)

In [15]:
premier_league=premier_league.reset_index()

In [17]:
premier_league.drop(columns='index',axis=1,inplace=True)

In [30]:
premier_league.sample(1).iloc[0]

Date                  2022-10-01
Time                       15:00
Comp              Premier League
Round                Matchweek 9
Day                          Sat
Venue                       Home
Result                         D
GF                             3
GA                             3
Opponent                Brighton
xG_x                         1.5
xGA                          1.4
Poss                        54.0
Attendance               53320.0
Captain         Jordan Henderson
Formation                  4-3-3
Referee              Andy Madley
Match Report        Match Report
Notes                        NaN
Gls                          2.0
Sh                          15.0
SoT                          7.0
SoT%                        46.7
G/Sh                        0.13
G/SoT                       0.29
FK                           1.0
PK                           0.0
PKatt                        0.0
xG_y                         1.5
npxG                         1.5
npxG/Sh   

In [19]:
premier_league.to_csv('premier_league.csv') 
premier_league.to_pickle('premier_league.pickle') 

In [10]:
# ['Gls','Sh','SoT','SoT%','G/Sh','G/SoT','Dist','FK','PK','PKatt','xG','npxG','npxG/Sh','G-xG','np:G-xG']